# TP2
1. Um  programa imperativo pode ser descrito por um modelo do tipo Control Flow Automaton (CFA) como ilustrado no exemplo seguinte

<img src="automato.png" alt="autómato" style="width: 600px;"/>

Este programa implementa a multiplicação de dois inteiros a,b fornecidos como “input”, e com precisão limitada a $\,n\,$ bits (fornecido como parâmetro do programa). 

Note-se que:

    - Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”. 
    - Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um output com as operações indicadas
    - Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo


a. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste autómato. Para isso identifique as variáveis do modelo, o estado inicial e a relação de transição.

    
b. Verifique se $$\;\;\mathsf{P}\,\equiv\,(x*y + z = a*b)\;\;$$é um invariante deste comportamento.

In [1]:
from pysmt.shortcuts import *
from z3 import *

In [2]:
# FOTS = (SMT, vetor_vars, init, trans)
# SMT - LIA
# vetor_vars = ["a", "b", z]

def declare(i, n):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = BitVec('x'+str(i), n)
    state['y'] = BitVec('y'+str(i), n)
    state['z'] = BitVec('z'+str(i), n)
    
    return state


In [3]:
def init(state, a, b):
    return And((state['pc']== 0), (state['x'] == a), (state['y'] == b), (state['z'] == 0))

In [4]:
def trans(curr, prox):
    t0 = And(
            (curr['pc'] == 0),
            (prox['pc'] ==1),
            (curr['x'] == prox['x']),
            (curr['y'] == prox['y']),
            (curr['z'] == prox['z']),
    )
    
    # y = 0
    t1 = And(
        (curr['y'] == 0),
        (curr['pc'] == 1),
        (prox['pc'] == 5),
        (curr['x'] == prox['x']),
        (curr['y'] == prox['y']),
        (curr['z'] == prox['z'])
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        (curr['y'] == 0),
        ((curr['y'] % 2) == 1),
        (curr['pc'] == 1),
        (prox['pc'] ==2),
        (curr['x'] == prox['x']),
        (prox['y'] == curr['y'] -1),
        (prox['z'] == curr['z']+curr['x'])
    )
    
    # y != 0 ^ even(y)
    t3 = And(
        (curr['y'] == 0),
        ((curr['y'] % 2) == 0),
        (curr['pc'] == 1),
        (prox['pc'] == 3),
        (prox['x'] == curr['x']*2),
        (prox['y'] == curr['y']/2),
        (prox['z'] == curr['z'])
    )
    
    # Q3 -> Q1
    t4 = And(
        (curr['pc'] == 3),
        (prox['pc'] == 1),
        (prox['x'] == curr['x']),
        (prox['y'] == curr['y']),
        (prox['z'] == curr['z'])
    )
        
    #Q2 -> Q1
    t5 = And(
        (curr['pc'] == 2),
        (prox['pc'] == 1),
        (prox['x'] == curr['x']),
        (prox['y'] == curr['y']),
        (prox['z'] == curr['z'])
    )
     
    # tbd
    #t6 =
    
    return Or(t0,t1, t2, t3, t4, t5)

In [6]:
def gera_traco(declare,init,trans,k):

    s = Solver()
    
    trace = [declare(i, 16) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],5,2))
        
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i+1]))
            
    if s.check():
        #m = s.model()
        #print(m)
        for i in range(k):
            print("Passo ", i)
            for v in trace[i]:
                print(v, "=", trace[i][v])
            print("----------------")
    else:
        print('e0sodonk')
                
gera_traco(declare,init,trans,20)

Passo  0
pc = pc0
x = x0
y = y0
z = z0
----------------
Passo  1
pc = pc1
x = x1
y = y1
z = z1
----------------
Passo  2
pc = pc2
x = x2
y = y2
z = z2
----------------
Passo  3
pc = pc3
x = x3
y = y3
z = z3
----------------
Passo  4
pc = pc4
x = x4
y = y4
z = z4
----------------
Passo  5
pc = pc5
x = x5
y = y5
z = z5
----------------
Passo  6
pc = pc6
x = x6
y = y6
z = z6
----------------
Passo  7
pc = pc7
x = x7
y = y7
z = z7
----------------
Passo  8
pc = pc8
x = x8
y = y8
z = z8
----------------
Passo  9
pc = pc9
x = x9
y = y9
z = z9
----------------
Passo  10
pc = pc10
x = x10
y = y10
z = z10
----------------
Passo  11
pc = pc11
x = x11
y = y11
z = z11
----------------
Passo  12
pc = pc12
x = x12
y = y12
z = z12
----------------
Passo  13
pc = pc13
x = x13
y = y13
z = z13
----------------
Passo  14
pc = pc14
x = x14
y = y14
z = z14
----------------
Passo  15
pc = pc15
x = x15
y = y15
z = z15
----------------
Passo  16
pc = pc16
x = x16
y = y16
z = z16
----------------
Passo  17
pc 